In [1]:
import argparse
import os
import numpy as np
import math

import torchvision.transforms as transforms
from torchvision.utils import save_image

from torch.utils.data import DataLoader
from torchvision import datasets
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [11]:
def weights_init_normal(m):
    classname = m.__class__.__name__
    if classname.find('Conv') != -1:
        torch.nn.init.normal_(m.weight.data, 0.0, 0.02)
    elif classname.find('BatchNorm2d') != -1:
        torch.nn.init.normal_(m.weight.data, 1.0, 0.02)
        torch.nn.init.constant_(m.bias.data, 0.0)

In [20]:
class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()

        self.label_emb = nn.Embedding(n_classes, latent_dim)

        self.init_size = img_size // 4 # Initial size before upsampling
        self.l1 = nn.Sequential(nn.Linear(latent_dim, 128*self.init_size**2))

        self.conv_blocks = nn.Sequential(
            nn.BatchNorm2d(128),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 128, 3, stride=1, padding=1),
            nn.BatchNorm2d(128, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Upsample(scale_factor=2),
            nn.Conv2d(128, 64, 3, stride=1, padding=1),
            nn.BatchNorm2d(64, 0.8),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Conv2d(64, channels, 3, stride=1, padding=1),
            nn.Tanh()
        )

    def forward(self, noise, labels):
        gen_input = torch.mul(self.label_emb(labels), noise)
        out = self.l1(gen_input)
        out = out.view(out.shape[0], 128, self.init_size, self.init_size)
        img = self.conv_blocks(out)
        return img

In [21]:
class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()

        def discriminator_block(in_filters, out_filters, bn=True):
            """Returns layers of each discriminator block"""
            block = [   nn.Conv2d(in_filters, out_filters, 3, 2, 1),
                        nn.LeakyReLU(0.2, inplace=True),
                        nn.Dropout2d(0.25)]
            if bn:
                block.append(nn.BatchNorm2d(out_filters, 0.8))
            return block

        self.conv_blocks = nn.Sequential(
            *discriminator_block(channels, 16, bn=False),
            *discriminator_block(16, 32),
            *discriminator_block(32, 64),
            *discriminator_block(64, 128),
        )

        # The height and width of downsampled image
        ds_size = img_size // 2**4

        # Output layers
        self.adv_layer = nn.Sequential( nn.Linear(128*ds_size**2, 1),
                                        nn.Sigmoid())
        self.aux_layer = nn.Sequential( nn.Linear(128*ds_size**2, n_classes),
                                        nn.Softmax())

    def forward(self, img):
        out = self.conv_blocks(img)
        out = out.view(out.shape[0], -1)
        validity = self.adv_layer(out)
        label = self.aux_layer(out)

        return validity, label

In [23]:
n_epochs = 20
batch_size = 64
lr = 0.0002
b1 = 0.5
b2 = 0.999
n_cpu = 1
latent_dim = 100
n_classes = 10
img_size =32
channels = 1
sample_interval = 400

In [24]:
cuda = True if torch.cuda.is_available() else False

# Loss functions
adversarial_loss = torch.nn.BCELoss()
auxiliary_loss = torch.nn.CrossEntropyLoss()

# Initialize generator and discriminator
generator = Generator()
discriminator = Discriminator()

if cuda:
    generator.cuda()
    discriminator.cuda()
    adversarial_loss.cuda()
    auxiliary_loss.cuda()

# Initialize weights
generator.apply(weights_init_normal)
discriminator.apply(weights_init_normal)

# Configure data loader
os.makedirs('images', exist_ok=True)
os.makedirs('mnist', exist_ok=True)
dataloader = torch.utils.data.DataLoader(
    datasets.MNIST('mnist', train=True, download=True,
                   transform=transforms.Compose([
                        transforms.Resize(img_size),
                        transforms.ToTensor(),
                        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
                   ])),
    batch_size=batch_size, shuffle=True)

# Optimizers
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

FloatTensor = torch.cuda.FloatTensor if cuda else torch.FloatTensor
LongTensor = torch.cuda.LongTensor if cuda else torch.LongTensor

In [25]:
def sample_image(n_row, batches_done):
    """Saves a grid of generated digits ranging from 0 to n_classes"""
    # Sample noise
    z = Variable(FloatTensor(np.random.normal(0, 1, (n_row**2, latent_dim))))
    # Get labels ranging from 0 to n_classes for n rows
    labels = np.array([num for _ in range(n_row) for num in range(n_row)])
    labels = Variable(LongTensor(labels))
    gen_imgs = generator(z, labels)
    save_image(gen_imgs.data, 'images/%d.png' % batches_done, nrow=n_row, normalize=True)

In [27]:

for epoch in range(n_epochs):
    for i, (imgs, labels) in enumerate(dataloader):

        batch_size = imgs.shape[0]

        # Adversarial ground truths
        valid = Variable(FloatTensor(batch_size, 1).fill_(1.0), requires_grad=False)
        fake = Variable(FloatTensor(batch_size, 1).fill_(0.0), requires_grad=False)

        # Configure input
        real_imgs = Variable(imgs.type(FloatTensor))
        labels = Variable(labels.type(LongTensor))

        # -----------------
        #  Train Generator
        # -----------------

        optimizer_G.zero_grad()

        # Sample noise and labels as generator input
        z = Variable(FloatTensor(np.random.normal(0, 1, (batch_size, latent_dim))))
        gen_labels = Variable(LongTensor(np.random.randint(0, n_classes, batch_size)))

        # Generate a batch of images
        gen_imgs = generator(z, gen_labels)

        # Loss measures generator's ability to fool the discriminator
        validity, pred_label = discriminator(gen_imgs)
        g_loss = 0.5 * (adversarial_loss(validity, valid) + auxiliary_loss(pred_label, gen_labels))

        g_loss.backward()
        optimizer_G.step()

        # ---------------------
        #  Train Discriminator
        # ---------------------

        optimizer_D.zero_grad()

        # Loss for real images
        real_pred, real_aux = discriminator(real_imgs)
        d_real_loss =  (adversarial_loss(real_pred, valid) + auxiliary_loss(real_aux, labels)) / 2

        # Loss for fake images
        fake_pred, fake_aux = discriminator(gen_imgs.detach())
        d_fake_loss =  (adversarial_loss(fake_pred, fake) + auxiliary_loss(fake_aux, gen_labels)) / 2

        # Total discriminator loss
        d_loss = (d_real_loss + d_fake_loss) / 2

        # Calculate discriminator accuracy
        pred = np.concatenate([real_aux.data.cpu().numpy(), fake_aux.data.cpu().numpy()], axis=0)
        gt = np.concatenate([labels.data.cpu().numpy(), gen_labels.data.cpu().numpy()], axis=0)
        d_acc = np.mean(np.argmax(pred, axis=1) == gt)

        d_loss.backward()
        optimizer_D.step()

        print ("[Epoch %d/%d] [Batch %d/%d] [D loss: %f, acc: %d%%] [G loss: %f]" % (epoch, n_epochs, i, len(dataloader),
                                                            d_loss.item(), 100 * d_acc,
                                                            g_loss.item()))
        batches_done = epoch * len(dataloader) + i
        if batches_done % sample_interval == 0:
            sample_image(n_row=10, batches_done=batches_done)

d:\anaconda3\envs\pytorch\lib\site-packages\torch\nn\modules\container.py:91: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  input = module(input)


[Epoch 0/20] [Batch 0/938] [D loss: 1.167612, acc: 74%] [G loss: 1.271133]
[Epoch 0/20] [Batch 1/938] [D loss: 1.152469, acc: 80%] [G loss: 1.251734]
[Epoch 0/20] [Batch 2/938] [D loss: 1.172235, acc: 81%] [G loss: 1.232967]
[Epoch 0/20] [Batch 3/938] [D loss: 1.143113, acc: 84%] [G loss: 1.251549]
[Epoch 0/20] [Batch 4/938] [D loss: 1.161315, acc: 77%] [G loss: 1.269115]
[Epoch 0/20] [Batch 5/938] [D loss: 1.116946, acc: 82%] [G loss: 1.292977]
[Epoch 0/20] [Batch 6/938] [D loss: 1.179958, acc: 77%] [G loss: 1.241820]
[Epoch 0/20] [Batch 7/938] [D loss: 1.135620, acc: 84%] [G loss: 1.171989]
[Epoch 0/20] [Batch 8/938] [D loss: 1.148141, acc: 78%] [G loss: 1.248136]
[Epoch 0/20] [Batch 9/938] [D loss: 1.156196, acc: 81%] [G loss: 1.187639]
[Epoch 0/20] [Batch 10/938] [D loss: 1.125762, acc: 85%] [G loss: 1.226063]
[Epoch 0/20] [Batch 11/938] [D loss: 1.182852, acc: 74%] [G loss: 1.262252]
[Epoch 0/20] [Batch 12/938] [D loss: 1.142672, acc: 85%] [G loss: 1.255781]
[Epoch 0/20] [Batch 13

[Epoch 0/20] [Batch 108/938] [D loss: 1.123557, acc: 78%] [G loss: 1.245754]
[Epoch 0/20] [Batch 109/938] [D loss: 1.118179, acc: 85%] [G loss: 1.251640]
[Epoch 0/20] [Batch 110/938] [D loss: 1.144415, acc: 85%] [G loss: 1.205819]
[Epoch 0/20] [Batch 111/938] [D loss: 1.091398, acc: 97%] [G loss: 1.187110]
[Epoch 0/20] [Batch 112/938] [D loss: 1.110569, acc: 87%] [G loss: 1.199159]
[Epoch 0/20] [Batch 113/938] [D loss: 1.105604, acc: 87%] [G loss: 1.241276]
[Epoch 0/20] [Batch 114/938] [D loss: 1.140315, acc: 85%] [G loss: 1.159075]
[Epoch 0/20] [Batch 115/938] [D loss: 1.097564, acc: 87%] [G loss: 1.194911]
[Epoch 0/20] [Batch 116/938] [D loss: 1.113307, acc: 84%] [G loss: 1.239000]
[Epoch 0/20] [Batch 117/938] [D loss: 1.148553, acc: 85%] [G loss: 1.210604]
[Epoch 0/20] [Batch 118/938] [D loss: 1.122137, acc: 82%] [G loss: 1.276798]
[Epoch 0/20] [Batch 119/938] [D loss: 1.139223, acc: 82%] [G loss: 1.266229]
[Epoch 0/20] [Batch 120/938] [D loss: 1.123197, acc: 92%] [G loss: 1.172797]

[Epoch 0/20] [Batch 215/938] [D loss: 1.102819, acc: 89%] [G loss: 1.223593]
[Epoch 0/20] [Batch 216/938] [D loss: 1.113724, acc: 90%] [G loss: 1.182641]
[Epoch 0/20] [Batch 217/938] [D loss: 1.135502, acc: 86%] [G loss: 1.199797]
[Epoch 0/20] [Batch 218/938] [D loss: 1.116303, acc: 90%] [G loss: 1.195095]
[Epoch 0/20] [Batch 219/938] [D loss: 1.096414, acc: 89%] [G loss: 1.217826]
[Epoch 0/20] [Batch 220/938] [D loss: 1.093616, acc: 91%] [G loss: 1.193087]
[Epoch 0/20] [Batch 221/938] [D loss: 1.106415, acc: 89%] [G loss: 1.256888]
[Epoch 0/20] [Batch 222/938] [D loss: 1.126501, acc: 88%] [G loss: 1.239052]
[Epoch 0/20] [Batch 223/938] [D loss: 1.106943, acc: 89%] [G loss: 1.179067]
[Epoch 0/20] [Batch 224/938] [D loss: 1.105641, acc: 89%] [G loss: 1.209171]
[Epoch 0/20] [Batch 225/938] [D loss: 1.105998, acc: 86%] [G loss: 1.222698]
[Epoch 0/20] [Batch 226/938] [D loss: 1.132900, acc: 88%] [G loss: 1.159772]
[Epoch 0/20] [Batch 227/938] [D loss: 1.114958, acc: 88%] [G loss: 1.252689]

[Epoch 0/20] [Batch 322/938] [D loss: 1.117952, acc: 88%] [G loss: 1.120356]
[Epoch 0/20] [Batch 323/938] [D loss: 1.109783, acc: 87%] [G loss: 1.231667]
[Epoch 0/20] [Batch 324/938] [D loss: 1.097534, acc: 82%] [G loss: 1.224063]
[Epoch 0/20] [Batch 325/938] [D loss: 1.103352, acc: 91%] [G loss: 1.113552]
[Epoch 0/20] [Batch 326/938] [D loss: 1.108103, acc: 87%] [G loss: 1.200061]
[Epoch 0/20] [Batch 327/938] [D loss: 1.125583, acc: 85%] [G loss: 1.164944]
[Epoch 0/20] [Batch 328/938] [D loss: 1.112078, acc: 88%] [G loss: 1.224772]
[Epoch 0/20] [Batch 329/938] [D loss: 1.109341, acc: 89%] [G loss: 1.124636]
[Epoch 0/20] [Batch 330/938] [D loss: 1.111248, acc: 85%] [G loss: 1.201753]
[Epoch 0/20] [Batch 331/938] [D loss: 1.111921, acc: 85%] [G loss: 1.202547]
[Epoch 0/20] [Batch 332/938] [D loss: 1.111934, acc: 91%] [G loss: 1.172875]
[Epoch 0/20] [Batch 333/938] [D loss: 1.111529, acc: 85%] [G loss: 1.173290]
[Epoch 0/20] [Batch 334/938] [D loss: 1.127071, acc: 88%] [G loss: 1.138810]

...

[Epoch 19/20] [Batch 465/938] [D loss: 1.064041, acc: 99%] [G loss: 1.061796]
[Epoch 19/20] [Batch 466/938] [D loss: 1.107569, acc: 100%] [G loss: 1.077246]
[Epoch 19/20] [Batch 467/938] [D loss: 1.104478, acc: 97%] [G loss: 1.082020]
[Epoch 19/20] [Batch 468/938] [D loss: 1.046494, acc: 98%] [G loss: 1.112635]
[Epoch 19/20] [Batch 469/938] [D loss: 1.091873, acc: 96%] [G loss: 1.056701]
[Epoch 19/20] [Batch 470/938] [D loss: 1.111246, acc: 98%] [G loss: 1.122453]
[Epoch 19/20] [Batch 471/938] [D loss: 1.105031, acc: 96%] [G loss: 1.123811]
[Epoch 19/20] [Batch 472/938] [D loss: 1.083460, acc: 99%] [G loss: 1.062028]
[Epoch 19/20] [Batch 473/938] [D loss: 1.052536, acc: 97%] [G loss: 1.119212]
[Epoch 19/20] [Batch 474/938] [D loss: 1.107828, acc: 95%] [G loss: 1.104912]
[Epoch 19/20] [Batch 475/938] [D loss: 1.085431, acc: 100%] [G loss: 1.078010]
[Epoch 19/20] [Batch 476/938] [D loss: 1.070810, acc: 98%] [G loss: 1.053670]
[Epoch 19/20] [Batch 477/938] [D loss: 1.092189, acc: 96%] [G 

[Epoch 19/20] [Batch 570/938] [D loss: 1.081431, acc: 98%] [G loss: 1.101761]
[Epoch 19/20] [Batch 571/938] [D loss: 1.087105, acc: 97%] [G loss: 1.076723]
[Epoch 19/20] [Batch 572/938] [D loss: 1.107136, acc: 96%] [G loss: 1.059445]
[Epoch 19/20] [Batch 573/938] [D loss: 1.077678, acc: 96%] [G loss: 1.054811]
[Epoch 19/20] [Batch 574/938] [D loss: 1.091825, acc: 97%] [G loss: 1.100905]
[Epoch 19/20] [Batch 575/938] [D loss: 1.083487, acc: 98%] [G loss: 1.105533]
[Epoch 19/20] [Batch 576/938] [D loss: 1.143614, acc: 97%] [G loss: 1.060561]
[Epoch 19/20] [Batch 577/938] [D loss: 1.064703, acc: 97%] [G loss: 1.113559]
[Epoch 19/20] [Batch 578/938] [D loss: 1.070500, acc: 96%] [G loss: 1.109375]
[Epoch 19/20] [Batch 579/938] [D loss: 1.062679, acc: 97%] [G loss: 1.055767]
[Epoch 19/20] [Batch 580/938] [D loss: 1.110357, acc: 98%] [G loss: 1.093438]
[Epoch 19/20] [Batch 581/938] [D loss: 1.067772, acc: 99%] [G loss: 1.085705]
[Epoch 19/20] [Batch 582/938] [D loss: 1.076839, acc: 96%] [G lo

[Epoch 19/20] [Batch 675/938] [D loss: 1.075412, acc: 99%] [G loss: 1.086762]
[Epoch 19/20] [Batch 676/938] [D loss: 1.062258, acc: 98%] [G loss: 1.061069]
[Epoch 19/20] [Batch 677/938] [D loss: 1.108708, acc: 98%] [G loss: 1.069941]
[Epoch 19/20] [Batch 678/938] [D loss: 1.064273, acc: 100%] [G loss: 1.029987]
[Epoch 19/20] [Batch 679/938] [D loss: 1.108423, acc: 99%] [G loss: 1.051504]
[Epoch 19/20] [Batch 680/938] [D loss: 1.113649, acc: 96%] [G loss: 1.102159]
[Epoch 19/20] [Batch 681/938] [D loss: 1.065928, acc: 100%] [G loss: 1.077991]
[Epoch 19/20] [Batch 682/938] [D loss: 1.104836, acc: 93%] [G loss: 1.067522]
[Epoch 19/20] [Batch 683/938] [D loss: 1.094037, acc: 98%] [G loss: 1.118999]
[Epoch 19/20] [Batch 684/938] [D loss: 1.087486, acc: 96%] [G loss: 1.104485]
[Epoch 19/20] [Batch 685/938] [D loss: 1.086985, acc: 98%] [G loss: 1.158245]
[Epoch 19/20] [Batch 686/938] [D loss: 1.108261, acc: 97%] [G loss: 1.070909]
[Epoch 19/20] [Batch 687/938] [D loss: 1.091453, acc: 98%] [G 

[Epoch 19/20] [Batch 780/938] [D loss: 1.084958, acc: 96%] [G loss: 1.109553]
[Epoch 19/20] [Batch 781/938] [D loss: 1.089014, acc: 98%] [G loss: 1.076503]
[Epoch 19/20] [Batch 782/938] [D loss: 1.065630, acc: 96%] [G loss: 1.116874]
[Epoch 19/20] [Batch 783/938] [D loss: 1.107098, acc: 97%] [G loss: 1.076003]
[Epoch 19/20] [Batch 784/938] [D loss: 1.064542, acc: 96%] [G loss: 1.082351]
[Epoch 19/20] [Batch 785/938] [D loss: 1.072924, acc: 96%] [G loss: 1.080274]
[Epoch 19/20] [Batch 786/938] [D loss: 1.089028, acc: 97%] [G loss: 1.083779]
[Epoch 19/20] [Batch 787/938] [D loss: 1.113725, acc: 97%] [G loss: 1.128601]
[Epoch 19/20] [Batch 788/938] [D loss: 1.086218, acc: 99%] [G loss: 1.144329]
[Epoch 19/20] [Batch 789/938] [D loss: 1.126271, acc: 96%] [G loss: 1.062625]
[Epoch 19/20] [Batch 790/938] [D loss: 1.078103, acc: 96%] [G loss: 1.099859]
[Epoch 19/20] [Batch 791/938] [D loss: 1.104776, acc: 96%] [G loss: 1.126635]
[Epoch 19/20] [Batch 792/938] [D loss: 1.090423, acc: 97%] [G lo

[Epoch 19/20] [Batch 886/938] [D loss: 1.087719, acc: 98%] [G loss: 1.056823]
[Epoch 19/20] [Batch 887/938] [D loss: 1.068274, acc: 97%] [G loss: 1.119822]
[Epoch 19/20] [Batch 888/938] [D loss: 1.108048, acc: 96%] [G loss: 1.100491]
[Epoch 19/20] [Batch 889/938] [D loss: 1.061541, acc: 99%] [G loss: 1.128425]
[Epoch 19/20] [Batch 890/938] [D loss: 1.097795, acc: 97%] [G loss: 1.072709]
[Epoch 19/20] [Batch 891/938] [D loss: 1.098293, acc: 99%] [G loss: 1.074040]
[Epoch 19/20] [Batch 892/938] [D loss: 1.111124, acc: 96%] [G loss: 1.101718]
[Epoch 19/20] [Batch 893/938] [D loss: 1.101970, acc: 99%] [G loss: 1.088724]
[Epoch 19/20] [Batch 894/938] [D loss: 1.096543, acc: 99%] [G loss: 1.098277]
[Epoch 19/20] [Batch 895/938] [D loss: 1.092435, acc: 98%] [G loss: 1.082257]
[Epoch 19/20] [Batch 896/938] [D loss: 1.085987, acc: 97%] [G loss: 1.104215]
[Epoch 19/20] [Batch 897/938] [D loss: 1.105408, acc: 97%] [G loss: 1.111559]
[Epoch 19/20] [Batch 898/938] [D loss: 1.108036, acc: 96%] [G lo